# Black Scholes Exercise 4: Numba implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [4]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext line_profiler

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)



The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# The Numba Black Scholes algorithm

What is different in the code?

In [5]:
import numba as nb
from math import log, sqrt, exp, erf

@nb.jit('(i8,f8[:],f8[:],f8[:],f8,f8,f8[:],f8[:])', nopython=True, cache=True)
def black_scholes( nopt, price, strike, t, rate, vol, call, put):
    mr = -rate
    sig_sig_two = vol * vol * 2

    for i in range(nopt):
        P = float( price[i] )
        S = strike [i]
        T = t [i]

        a = log(P / S)
        b = T * mr

        z = T * sig_sig_two
        c = 0.25 * z
        y = 1./sqrt(z)

        w1 = (a - b + c) * y 
        w2 = (a - b - c) * y

        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)

        Se = exp(b) * S

        call [i] = P * d1 - Se * d2
        put [i] = call [i] - P + Se


## Run timeit, cProfile, and line_profiler to see what is happening

In [6]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

11.5 ms ± 667 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         4 function calls in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.013    0.013    0.013    0.013 <ipython-input-5-b73323227477>:4(black_scholes)
        1    0.000    0.000    0.013    0.013 <string>:1(<module>)
        1    0.000    0.000    0.013    0.013 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [8]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

## Variant 2

What changed in the code from the previous one to this one?

In [9]:
def black_scholes_numba_opt(price, strike, t, mr, sig_sig_two):
        P = price
        S = strike
        T = t
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = 1./sqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        r  = P * d1 - Se * d2
        return complex(r, r - P + Se)


In [10]:
black_scholes_numba_opt_vec = nb.vectorize('c16(f8,f8,f8,f8,f8)', nopython=True)(black_scholes_numba_opt)

@nb.jit
def black_scholes(nopt, price, strike, t, rate, vol):
    sig_sig_two = vol*vol*2
    mr = -rate
    black_scholes_numba_opt_vec(price, strike, t, mr, sig_sig_two)

## Run timeit, cProfile, and line_profiler to see what is happening

In [11]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

11.8 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         4 function calls in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.013    0.013    0.013    0.013 <ipython-input-10-83ef3d26a385>:3(black_scholes)
        1    0.001    0.001    0.013    0.013 <string>:1(<module>)
        1    0.000    0.000    0.013    0.013 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [14]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Variant 3

What does this change?

In [15]:
black_scholes_numba_opt_vec = nb.vectorize('c16(f8,f8,f8,f8,f8)', target="parallel")(black_scholes_numba_opt)

@nb.jit
def black_scholes(nopt, price, strike, t, rate, vol):
    sig_sig_two = vol*vol*2
    mr = -rate
    black_scholes_numba_opt_vec(price, strike, t, mr, sig_sig_two)

## Run timeit, cProfile, and line_profiler to see what is happening

In [16]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

6.22 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         4 function calls in 0.009 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.009    0.009    0.009    0.009 <ipython-input-15-3caac9f2da84>:3(black_scholes)
        1    0.000    0.000    0.009    0.009 <string>:1(<module>)
        1    0.000    0.000    0.009    0.009 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [18]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)